In [1]:
import ee
ee.Initialize()

In [2]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
from sixs_emulator_ee_sentinel2_batch import SixS_emulator
from atmcorr_input import Atmcorr_input
from atmospheric_correction import atmospheric_correction
from radiance import radiance_from_TOA
from interpolated_LUTs import Interpolated_LUTs

ImportError: No module named 'sixs_emulator_ee_sentinel2_batch'

In [ ]:
# a place and a mission
geom = ee.Geometry.Point(107.65915, 16.31684)
mission = 'COPERNICUS/S2'  

In [ ]:
# 6S emulator
se = SixS_emulator(mission)

In [ ]:
# image collection
ic = ee.ImageCollection(mission)\
    .filterBounds(geom)\
    .filterDate('2017-02-01','2017-05-01')\
    .filter(ee.Filter.lt('MEAN_SOLAR_ZENITH_ANGLE',75))

In [ ]:
# iLUTs instance
iLUTs = Interpolated_LUTs('COPERNICUS/S2') # i.e. Sentinel2 

In [ ]:
# if this is first time you might have to download the look up tables
iLUTs.download_LUTs()

In [ ]:
# and run the interpolation
iLUTs.interpolate_LUTs()

If you are running this notebook in a docker container then you can save these interpolated look-up tables (and your Earth Engine authentication) for later using a [docker commit](https://github.com/samsammurphy/gee-atmcorr-S2-6SE/wiki/docker-commits). This will save them to memory so that you only have to do the set-up once.

In [ ]:
# otherwise, you can just load iLUTs from file
se.iLUTs = iLUTs.get()

In [ ]:
# extract atmcorr inputs as feature collection
Atmcorr_input.geom = geom  # specify target location (would use image centroid otherwise)
atmcorr_inputs = ic.map(Atmcorr_input.extractor).getInfo()
features = atmcorr_inputs['features']

In [ ]:
# atmospherically correct image collection

ic_atmospherically_corrected = []

for feature in features:

    # at-sensor radiance 
    toa = ee.Image(mission+'/'+feature['properties']['imgID'])
    rad = radiance_from_TOA(toa, feature)

    # 6S emulator
    cc = se.run(feature['properties']['atmcorr_inputs'])

    # atmospheric correction
    SR = atmospheric_correction(rad, cc)
    ic_atmospherically_corrected.append(SR)

In [ ]:
# Earth Engine image collection
ic_atmospherically_corrected = ee.ImageCollection(ic_atmospherically_corrected)\
.cast({'B10':'float', 'B11':'float', 'B12':'float', 'B8A':'float', 'B1':'float', 'B2':'float', 'B3':'float',
       'B4':'float', 'B5':'float', 'B6':'float','B7':'float', 'B8':'float', 'B9':'float'}, 
      ['B10', 'B11', 'B12', 'B8A', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9'])
#print (ic_atmospherically_corrected.getInfo())

In [ ]:
# test
SR = ee.Image(ic_atmospherically_corrected.first())
#print(SR.reduceRegion(ee.Reducer.mean(),geom).getInfo())

In [ ]:
# original image (top of atmosphere)
TOA = ee.Image(ic.mosaic()).divide(10000)

# surface reflectance (bottom of atmosphere)
SR = ee.Image(ic_atmospherically_corrected.mosaic())

from IPython.display import display, Image

region = geom.buffer(5000).bounds().getInfo()['coordinates']

before = Image(url=TOA.select(['B4','B3','B2']).getThumbUrl({
                'region':region,
                'min':0,
                'max':0.25,
                'gamma':1.5
                }))

after = Image(url=SR.select(['B4','B3','B2']).getThumbUrl({
                'region':region,
                'min':0,
                'max':0.25,
                'gamma':1.5
                }))

display(before, after)

test = ee.Image(ic_atmospherically_corrected.median())
dis_test = Image(url=test.select(['B11','B8','B4']).getThumbUrl({
'region':region,
'min':0.05,
'max':0.6,
'gamma':1
}))
display('Ra di', dis_test)